In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [8]:
def get_title(soup):

    
    try:
        title = soup.find("span", attrs={"id":'productTitle'})
        
        title_value = title.text

        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string


def get_price(soup):

    try:
        price = soup.find("span", attrs={'class':'a-offscreen'}).string.strip()

    except AttributeError:

        try:
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""

    return rating

def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""

    return review_count


In [ ]:

if __name__ == '__main__':
    
    for i in range(1,21):
    
        HEADERS = ({'User-Agent' : 'Mozilla/5.0 (Windows NT 10.4; x64; en-US) AppleWebKit/601.48 (KHTML, like Gecko) Chrome/49.0.2767.228 Safari/602.0 Edge/13.57484' , 'Accept-Language' : 'en-Us, en;q=0.5'})

        URL = f"https://www.amazon.in/s?k=bags&page={i}&crid=2M096C61O4MLT&qid=1692721033&sprefix=ba%2Caps%2C283&ref=sr_pg_2"

        webpage = requests.get(URL, headers=HEADERS)

        soup = BeautifulSoup(webpage.content, "html.parser")

        links = soup.find_all("a", attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

        links_list = []

        for link in links:
                links_list.append(link.get('href'))

        d = {"URL":[],"title":[], "price":[], "rating":[], "reviews":[]}

        for link in links_list:
            new_webpage = requests.get("https://www.amazon.in" + link, headers=HEADERS)
            new_soup = BeautifulSoup(new_webpage.content, "html.parser")
            d['URL'].append(new_webpage)
            d['title'].append(get_title(new_soup))
            d['price'].append(get_price(new_soup))
            d['rating'].append(get_rating(new_soup))
            d['reviews'].append(get_review_count(new_soup))

        amazon_df = pd.DataFrame.from_dict(d)
        amazon_df['title'].replace('', np.nan, inplace=True)
        amazon_df = amazon_df.dropna(subset=['title'])
        amazon_df.to_csv("amazon_bags_data.csv", header=True, index=False)


In [ ]:
amazon_df